In [1]:
print("⏳ Setting up environment for HMTAS inference...")

# --- Step 1: Ensure clean base ---
!pip uninstall -y bleurt tensorflow tensorflow-text -q || true
!rm -rf /root/.cache/pip /root/.cache/huggingface /root/.cache/nltk

# --- Step 2: Install core stable packages (quietly) ---
!pip install -q -U numpy==1.26.4 scikit-learn==1.3.2
!pip install -q transformers==4.41.2 datasets==2.18.0 sentence-transformers==2.7.0
!pip install -q rouge-score==0.1.2 bert-score==0.3.13
!pip install -q networkx==3.2.1 hdbscan==0.8.33 umap-learn==0.5.5
!pip install -q matplotlib seaborn nltk

# --- Step 3: Torch (CPU) install (GPU uses default runtime CUDA) ---
!pip install -q torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121

# --- Step 4: Download NLTK resources silently ---
import nltk
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

print("✅ Environment ready for HMTAS inference and visualization.")


⏳ Setting up environment for HMTAS inference...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 113.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.3.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [2]:
# hmtas_compare_kaggle.py
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import time, gc, json, random
from pathlib import Path
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rouge_score import rouge_scorer
from bert_score import score as bert_score_calculator
import nltk
from nltk.tokenize import sent_tokenize

# Optional metrics
try:
    import sacrebleu
    _HAS_SACREBLEU = True
except Exception:
    _HAS_SACREBLEU = False
try:
    from nltk.translate.meteor_score import meteor_score
    _HAS_METEOR = True
except Exception:
    _HAS_METEOR = False

import hdbscan
import networkx as nx

# -----------------------------
# Config
# -----------------------------
class Config:
    total_test_samples = 100  # reduced for speed
    ratios = {'cnn_dailymail::3.0.0': 0.35, 'multi_news': 0.35, 'xsum': 0.3}

    backbone = 'facebook/bart-large-cnn'
    max_input_length = 1024
    max_target_length = 150

    token_budget = 420
    min_cluster_size = 3
    per_cluster_topk = 3
    fallback_topk = 10
    sim_threshold = 0.15
    pagerank_alpha = 0.85
    pagerank_max_iter = 100

    num_beams = 6
    length_penalty = 2.0
    min_length = 60
    repetition_penalty = 1.1
    no_repeat_ngram_size = 3

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    seed = 42
    output_dir = Path('./hmtas_output')

config = Config()

# -----------------------------
# Utilities
# -----------------------------
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def mkdir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

# -----------------------------
# Encoder and clustering
# -----------------------------
class SentenceEncoderGPU:
    def __init__(self, device):
        self.device = device
        self.model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
        self.model.eval()
    def encode(self, sents):
        if not sents:
            return torch.empty(0, 384, device=self.device)
        return self.model.encode(sents, convert_to_tensor=True, device=self.device, show_progress_bar=False)
    def cosine(self, a, b):
        a, b = F.normalize(a, p=2, dim=-1), F.normalize(b, p=2, dim=-1)
        return torch.mm(a, b.t())

class HMTASClusterer:
    def __init__(self, cfg): self.cfg = cfg
    def cluster(self, emb):
        e = emb.detach().cpu().numpy()
        n = e.shape[0]
        if n < self.cfg.min_cluster_size:
            return np.zeros(n, dtype=int)
        c = hdbscan.HDBSCAN(min_cluster_size=self.cfg.min_cluster_size, min_samples=1)
        labels = c.fit_predict(e)
        if (labels == -1).any():
            non_noise = np.where(labels != -1)[0]
            for i in np.where(labels == -1)[0]:
                sims = F.cosine_similarity(torch.tensor(e[i]).unsqueeze(0), torch.tensor(e[non_noise]), dim=-1)
                labels[i] = int(labels[non_noise][sims.argmax().item()])
        return labels

# -----------------------------
# HMTAS processor
# -----------------------------
class HMTASProcessor:
    def __init__(self, cfg, tokenizer):
        self.cfg = cfg
        self.tok = tokenizer
        self.encoder = SentenceEncoderGPU(cfg.device)
        self.clusterer = HMTASClusterer(cfg)
    def preprocess(self, docs):
        sents = []
        for d in docs:
            sents.extend([s for s in sent_tokenize(d) if 8 < len(s.split()) < 150])
        return sents
    def create_guided_input(self, docs):
        sents = self.preprocess(docs)
        if not sents: return " ".join(f"[DOC_{i}] {d}" for i,d in enumerate(docs))
        emb = self.encoder.encode(sents)
        sim = self.encoder.cosine(emb, emb)
        labels = self.clusterer.cluster(emb)
        chosen = []
        for l in np.unique(labels):
            idxs = np.where(labels == l)[0].tolist()
            sub = sim[idxs][:, idxs].cpu().numpy()
            sub[sub < self.cfg.sim_threshold] = 0
            np.fill_diagonal(sub, 0)
            G = nx.from_numpy_array(sub)
            pr = nx.pagerank(G, alpha=self.cfg.pagerank_alpha)
            ranked = sorted(pr.items(), key=lambda x: x[1], reverse=True)
            chosen.extend([idxs[i] for i,_ in ranked[:self.cfg.per_cluster_topk]])
        chosen = list(dict.fromkeys(chosen))
        if len(chosen) < self.cfg.fallback_topk:
            sums = sim.sum(dim=1).cpu().numpy()
            topk = np.argsort(-sums)[:self.cfg.fallback_topk]
            chosen = list(dict.fromkeys(chosen + topk.tolist()))
        # Token budget
        total, final = 0, []
        for i in chosen:
            n = len(self.tok.encode(sents[i], add_special_tokens=False))
            if total + n > self.cfg.token_budget: break
            final.append(i); total += n
        guide = " ".join([sents[i] for i in final])
        doc_join = " ".join(f"[DOC_{i}] {d}" for i,d in enumerate(docs))
        return f"Summarize the following multi-document input:\n[GUIDANCE] {guide}\n[DOCUMENTS] {doc_join}"

# -----------------------------
# Evaluation helpers
# -----------------------------
def compute_metrics(refs, cands, srcs, tok):
    rouge = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)
    r1,r2,rL,bleus,meteors,covs,comps = [],[],[],[],[],[],[]
    for r,c,s in zip(refs,cands,srcs):
        sc = rouge.score(r,c)
        r1.append(sc['rouge1'].fmeasure); r2.append(sc['rouge2'].fmeasure); rL.append(sc['rougeL'].fmeasure)
        cov,comp = coverage_compression(s,c,tok)
        covs.append(cov); comps.append(comp)
        if _HAS_SACREBLEU:
            bleus.append(sacrebleu.sentence_bleu(c,[r]).score)
        if _HAS_METEOR:
            try: meteors.append(meteor_score([r],c)*100)
            except: meteors.append(0)
    try:
        _,_,F1 = bert_score_calculator(cands, refs, lang='en', model_type='distilbert-base-uncased', device=config.device)
        bertF = (F1.cpu().numpy()*100).tolist()
    except: bertF = [0.0]*len(cands)
    return dict(rouge1=r1, rouge2=r2, rougeL=rL, bertscore_f1=bertF, bleu=bleus, meteor=meteors, coverage=covs, compression=comps)

def coverage_compression(src, summ, tok):
    src_t = len(tok.encode(src, add_special_tokens=False))
    sum_t = len(tok.encode(summ, add_special_tokens=False))
    if src_t==0: return 0,0
    return (sum_t/src_t)*100, src_t/max(sum_t,1)

def batch_generate(model, tok, texts):
    outs=[]
    for i in range(0,len(texts),2):
        b=texts[i:i+2]
        enc=tok(b,max_length=config.max_input_length,truncation=True,padding=True,return_tensors='pt').to(config.device)
        with torch.autocast(device_type='cuda',dtype=torch.float16):
            with torch.no_grad():
                ids=model.generate(enc.input_ids,attention_mask=enc.attention_mask,
                                   num_beams=config.num_beams,max_length=config.max_target_length,
                                   min_length=config.min_length,length_penalty=config.length_penalty,
                                   repetition_penalty=config.repetition_penalty,
                                   no_repeat_ngram_size=config.no_repeat_ngram_size)
        outs.extend(tok.batch_decode(ids,skip_special_tokens=True))
    return [o.strip() for o in outs]

# -----------------------------
# Load multi-document test set
# -----------------------------
def load_multi_document_dataset(cfg):
    print("Loading multi-document subset...")
    ds1 = load_dataset('cnn_dailymail','3.0.0',split='test[:1000]')
    ds2 = load_dataset('multi_news',split='test[:1000]')
    ds3 = load_dataset('xsum',split='test[:1000]')
    pool=[]
    for i in range(cfg.total_test_samples):
        docs = []
        docs.append(ds1[i]['article'])
        docs.append(ds2[i]['document'])
        docs.append(ds3[i]['document'])
        summary = ds2[i]['summary']
        pool.append({'texts':docs,'summary':summary})
    print(f"✓ Built {len(pool)} multi-document examples")
    return pool

# -----------------------------
# Main
# -----------------------------
def main():
    set_seed(config.seed)
    mkdir(config.output_dir)
    print("="*80)
    print("🚀 HMTAS — Hierarchical Multi-Document Text Abstractive Summarization")
    print("="*80)
    print(f"Device: {config.device}")
    print(f"Samples: {config.total_test_samples}")
    print("="*80)

    test_pool = load_multi_document_dataset(config)
    tok = AutoTokenizer.from_pretrained(config.backbone)
    proc = HMTASProcessor(config, tok)
    model = AutoModelForSeq2SeqLM.from_pretrained(config.backbone).to(config.device)

    # --- Evaluate HMTAS ---
    print("\nEvaluating HMTAS (guided multi-doc)...")
    t0=time.time()
    guided_inputs=[proc.create_guided_input(d['texts']) for d in test_pool]
    hmtas_summ=batch_generate(model,tok,guided_inputs)
    t_hmtas=time.time()-t0
    refs=[d['summary'] for d in test_pool]
    srcs=[" ".join(d['texts']) for d in test_pool]
    res_hmtas=compute_metrics(refs,hmtas_summ,srcs,tok)

    # --- Evaluate Base ---
    print("\nEvaluating Base (flat multi-doc)...")
    t1=time.time()
    flat_inputs=[" ".join(d['texts']) for d in test_pool]
    base_summ=batch_generate(model,tok,flat_inputs)
    t_base=time.time()-t1
    res_base=compute_metrics(refs,base_summ,srcs,tok)

    # --- Compare ---
    def mavg(x): return float(np.mean(x)*100)
    print("\n================= RESULTS =================")
    print("| Metric | HMTAS | Base | Win% |")
    print("|---------|-------|------|------|")
    for k in ['rouge1','rouge2','rougeL','bertscore_f1']:
        a,b=np.array(res_hmtas[k]),np.array(res_base[k])
        win=(a>b).sum()/len(a)*100
        print(f"| {k:<9} | {np.mean(a)*100:5.2f} | {np.mean(b)*100:5.2f} | {win:4.1f}% |")
    print(f"\nBLEU: {np.mean(res_hmtas['bleu']):.2f} vs {np.mean(res_base['bleu']):.2f}")
    print(f"METEOR: {np.mean(res_hmtas['meteor']):.2f} vs {np.mean(res_base['meteor']):.2f}")
    print(f"Time(s): HMTAS {t_hmtas:.1f} | Base {t_base:.1f}")

    # --- Save ---
    df=pd.DataFrame({
        'idx':range(len(refs)),
        'reference':refs,
        'source':srcs,
        'hmtas_summary':hmtas_summ,
        'base_summary':base_summ
    })
    df.to_csv(config.output_dir/'summaries.csv',index=False)
    model.save_pretrained(config.output_dir/'HMTAS_backbone_saved')
    tok.save_pretrained(config.output_dir/'HMTAS_backbone_saved')
    print(f"\n✓ Saved outputs and model to {config.output_dir}")

if __name__ == '__main__':
    main()


🚀 HMTAS — Hierarchical Multi-Document Text Abstractive Summarization
Device: cuda
Samples: 100
Loading multi-document subset...


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]


Evaluating Base (flat multi-doc)...


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}



================= RESULTS =================
| Metric | HMTAS | Base | Win% |
|---------|-------|------|------|
| rouge1    | 16.99 | 14.74 | 68.0% |
| rouge2    |  3.00 |  1.95 | 50.0% |
| rougeL    | 10.55 |  9.13 | 69.0% |
| bertscore_f1 | 7064.70 | 6973.63 | 62.0% |

BLEU: nan vs nan
METEOR: 0.00 vs 0.00
Time(s): HMTAS 120.1 | Base 102.7

✓ Saved outputs and model to hmtas_output
